In [ ]:
from agentlab.experiments.exp_utils import  RESULTS_DIR
from agentlab.analyze import inspect_results
from agentlab.analyze import ICML_2024_results
from pathlib import Path
import pandas as pd
RESULTS_DIR = Path("/mnt/colab_public/ui_copilot/ICML_2024/results")

%load_ext autoreload
%autoreload 2

## ablation study miniwob

In [ ]:
RESULTS_DIR = Path("/mnt/colab_public/ui_copilot/ICML_2024/results")
result_dir = RESULTS_DIR / "2024-02-01_03-24-01_ablation_study_browsergym_miniwob"
result_df = inspect_results.load_result_df(result_dir, set_index=False)
miniwob_dict = inspect_results.split_by_key(result_df, key="agent_args.chat_model_args.model_name")

In [ ]:
for model_name, result_df in miniwob_dict.items():
    print(model_name)
    report = inspect_results.global_report(result_df, reduce_fn=inspect_results.summarize)
    for idx in report.index:
        print(idx)
    inspect_results.display_report(report)
    print(report.index[0])

In [ ]:
ICML_2024_results.make_joint_ablation_study(miniwob_dict)

### OSS Models

In [ ]:
RESULTS_DIR = Path("/mnt/colab_public/ui_copilot/ICML_massimo")


result_df_list = []
for exp_name in ("2024-01-31_01-03-54_miniwob_random_search_prompt_OSS_LLMs", "2024-01-31_20-11-57_miniwob_random_search_prompt_OSS_LLMs"):
    result_dir = RESULTS_DIR / exp_name
    result_df = inspect_results.load_result_df(result_dir, set_index=False)
    result_df_list.append(result_df)

# concat the dataframes
result_df = pd.concat(result_df_list, ignore_index=True)

In [ ]:

inspect_results.set_index_from_variables(result_df, index_black_list=("*args.ace", "*workspace", "*tokens", "*url"))
miniwob_dict["CodeLlama"] = result_df
report = inspect_results.global_report(result_df, reduce_fn=inspect_results.summarize)
inspect_results.display_report(report)
print(report.index[1])

In [ ]:
for key, df in miniwob_dict.items():
    miniwob_dict[key] = ICML_2024_results.add_web_gum_subset(df)

In [ ]:
agent_dict = {"CodeLlama": (True, False, False, False, True, True, True, False, False, False, 'python', True, False),
              "openai/gpt-4-1106-preview": (True, True, True, 'bid+coord', False, 'box'),
              "openai/gpt-3.5-turbo-1106": (True, True, True, 'bid', False, 'center')}
report = ICML_2024_results.make_joint_report(miniwob_dict, agent_dict, use_category=True)
report

## ablation study workarena

In [ ]:
RESULTS_DIR = Path("/mnt/colab_public/ui_copilot/ICML_2024/results")

result_dir = RESULTS_DIR / "2024-02-01_03-20-14_ablation_study_browsergym_workarena"
result_df = inspect_results.load_result_df(result_dir, set_index=False)
workarena_dict = inspect_results.split_by_key(result_df, key="agent_args.chat_model_args.model_name")

In [ ]:

ICML_2024_results.make_joint_ablation_study(workarena_dict)

In [ ]:
for model_name, result_df in workarena_dict.items():
    print(model_name)

    report = inspect_results.global_report(result_df, reduce_fn=inspect_results.summarize)
    for idx in report.index:
        print(idx)
    inspect_results.display_report(report)
    

In [ ]:
agent_dict = {
    "openai/gpt-4-1106-preview": (True, False, 'bid', False, 'False'),
    "openai/gpt-3.5-turbo-1106": (True, True, 'bid', False, 'False'),
}
ICML_2024_results.make_joint_report(workarena_dict, agent_dict)

## Ablation Study WebArena


In [ ]:
RESULTS_DIR = Path("/mnt/colab_public/ui_copilot/ICML_2024_rebutal")
result_dir = RESULTS_DIR / "2024-03-20_14-02-09_maxime_gpt4_webarena"
print(result_dir)
result_df = inspect_results.load_result_df(result_dir, set_index=False)
webarena_dict = inspect_results.split_by_key(result_df, key="agent_args.chat_model_args.model_name")

In [ ]:
for model_name, result_df in webarena_dict.items():
    print(model_name)
    report = inspect_results.global_report(result_df, reduce_fn=inspect_results.summarize)
    for idx in report.index:
        print(idx)
    inspect_results.display_report(report)
    print(report.index[0])

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
reduce = False

info = {
    "miniwob": (
        miniwob_dict["openai/gpt-4-1106-preview"],
        (False, False, False, 'bid', False, 'False'),
        (True, True, True, 'bid+coord', True, 'box'),
    ),
    "workarena": (
        workarena_dict["openai/gpt-4-1106-preview"],
        (True, False, "bid", False, "False"),
        (True, True, "bid+coord", True, "box"),
    ),
    # "workarena": (
    #     workarena_dict["openai/gpt-4-1106-preview"],
    #     "GenericAgent",
    #     "GenericAgent",
    # ),
    "webarena": (
        webarena_dict["openai/gpt-4-0125-preview"],
        "GenericAgent",
        "GenericAgent",
    ),
}
reports = []

for benchmark, (df, base_agent, full_agent) in info.items():
    # remove miniwob.stock-market
    if benchmark == "miniwob":
        # remove stock-market, but using the task_name in the index
        df = df[df.index.get_level_values("task_name") != "miniwob.stock-market"]
        # df = df[df["task_name"] != "miniwob.stock-market"]

    for i, agent in enumerate((base_agent, full_agent)):
        df_agent = ICML_2024_results.select_single_agent(df, agent)
        if benchmark == "workarena":
            df_agent = ICML_2024_results.set_task_category_as_index(df_agent)
        # df_agent = ICML_2024_results.set_task_category_as_index(df_agent)
        if reduce:
            report = inspect_results.global_report(df_agent, reduce_fn=ICML_2024_results.paper_stats)
            # if benchmark == "miniwob" or benchmark == "webarena":
            report = report[report.index == "[ALL TASKS]"]
        else:
            report = df_agent
        report["agent"] = ["base_agent", "full_agent"][i]
        report["benchmark"] = benchmark
        
        reports.append(report)
report = pd.concat(reports)


# report.set_index("benchmark", inplace=True)
report = report[report.agent=="base_agent"]

# save the report to a file
report.to_csv(Path.home()/"token_stats_report.csv")

report

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
box_plot = False

# Set the fancy theme
# sns.set_theme(style="whitegrid")
sns.set_style("darkgrid")

df = report.rename_axis('new_index').reset_index(inplace=False)

df_melted = pd.melt(df, id_vars=['benchmark'], 
                    value_vars=['stats.max_token_pruned_html', 'stats.max_token_axtree_txt'],
                    var_name='Tokens', value_name='max tokens per episode')

plt.figure(figsize=(10, 6))  
plt.yscale('log')
if box_plot:
    sns.boxplot(data=df_melted, x='Tokens', y='max tokens per episode', hue='benchmark')
else:
    filtered_df = df_agent[df_agent["agent"] == "base_agent"]
    sns.violinplot(data=df_melted, x='Tokens', y='max tokens per episode', hue='benchmark', split=False)


plt.legend( loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=len(df_melted['benchmark'].unique()))

plt.xlabel('')
plt.show()



In [ ]:
# hide a subset of experiments based on a filter
################################################

from agentlab.experiments.exp_utils import hide_some_exp, ExpResult

# def filter(exp: ExpResult):
#     model_name = exp.exp_args.agent_args.chat_model_args.model_name
#     return "vision" in model_name

def filter(exp: ExpResult):
    flags = exp.exp_args.agent_args.flags
    return flags.action_space == "coord" or flags.action_space == "python"

result_dir = RESULTS_DIR / "2024-01-30_23-00-34_ablation_study_browsergym"

filtered_out = hide_some_exp(result_dir, filter, just_test=True)
len(filtered_out)

## Action Count

In [ ]:
RESULTS_DIR = Path("/mnt/colab_public/ui_copilot/ICML_2024_rebutal")
result_dir = RESULTS_DIR / "2024-03-20_14-02-09_maxime_gpt4_webarena"

action_count_webarena = ICML_2024_results.get_all_action_count(result_dir)
action_count_webarena.n_actions.hist(bins=100)

In [ ]:
RESULTS_DIR = Path("/mnt/colab_public/agentlab/ICML_2024/results")
result_dir = RESULTS_DIR / "2024-02-01_03-20-14_ablation_study_browsergym_workarena"

action_count_workarena = ICML_2024_results.get_all_action_count(result_dir)
action_count_workarena.n_actions.hist(bins=100)

In [ ]:

def n_action_stats(df, group_by="task_name"):

    def percentile_95(x):
        return x.quantile(0.95)

    if group_by is not None:
        df = df.groupby('task_name')

    return df.agg({'n_actions': ['min', 'max', 'mean', 'median', percentile_95]})


n_action_stats(action_count_workarena)

In [ ]:
print("WorkArena Action count stats")
display(n_action_stats(action_count_workarena, group_by=None))
print("WebArena Action count stats")
display(n_action_stats(action_count_webarena, group_by=None))